Latex macros
$
\newcommand{\R}{\mathbb{R}}
\newcommand{\C}{\mathbb{C}}
\newcommand{\bfb}{\mathbf{b}}
\newcommand{\bfr}{\mathbf{r}}
%
\newcommand{\bfA}{\mathbf{A}}
\newcommand{\bfQ}{\mathbf{Q}}
%
\newcommand{\HP}{\mathcal{C}}
\newcommand{\HM}{\mathcal{B}}
%
\newcommand{\ket}[1]{\left\vert #1 \right\rangle}
\newcommand{\bra}[1]{\left\langle #1 \right\vert}
\newcommand{\braket}[1]{\left\langle #1 \right\rangle}
\newcommand{\pmat}[1]{\begin{pmatrix}#1\end{pmatrix}}
$

# Use Case: Optimization of Vehicle Routing for a PickUpDeliveryProblem  <a class="tocSkip"/>
Dr. Florian Knäble, Fraunhofer IAO

# Step 1: Find an appropriate problem

**The Vehicle Routing Problem for a PickUpDelivery Problem with time windows**

- looking for an optimal subdivision of a given number of pickup and delivery tours for a given number of trucks 
- restrictions on daily driving times for every single truck
- restrictions on time windows outside which pickups or deliveries cannot be made
- time horizon for a whole week


## Reduction a to proof-of-concept model

- no time restrictions whatsoever
- number of tours and trucks must be kept minimal

**Given**
- number of tours (each with a pickup and with a delivery location)
- number of trucks (each with their own depot where they have to start and end their journey)

**Aim**
- minimize distances driven outside of actual PickUp-Delivery runs (those cannot be helped anyhow)   "Leerkilometer minimieren" 
- all PickUp-Delivery tours must be served
- trucks must start and end at depots

**Example with two trucks**


<img src="images/Tourplanning_Problem.PNG" />

- Depots are given in green and red
- Attention: each tour (with actually two locations is represented by a single location)
- This abstraction is possible, but the distances between tours are then not symmetric anymore 

**Distance Matrix**

- distances between all possible locations are noted down
- as said above, these are not necessarily symmetric
- locations are given by coordinates and distances are calculated by a suitable metric (e.g. $l_1$ or $l_2$)  

<img src="images/distance__matrix.PNG" />

The Cost function ("Leerkilometer") is calculated by adding up the costs of all connecting rides:

<img src="images/cost__function.PNG" />

**Binary Encoding**
- For the Quantum Computer we have to cast our problem into the form of a binary quadratic problem.
- Therefore we represent the order in which tours are visited by a permutation matrix 
(which has per definition binary entries)
- To encode the order of $n$ tours for one truck, we need then $n^2$ Qubits
- In the case of several trucks this number is multiplied with the number of trucks (we need a full permutation matrix for each truck)
- thus if $X$ is the permutation matrix. then $x_{ij}=1$ would mean that tour number $i$ is served at position $j$. 

<img src="images/permutation_matrix.PNG" />

**Quadratic Problem**
- in this encoding we can represent the cost function $f$ to be minimized by a quadratic function in the binary variables 
- we also need restrictions to ensure that all tours are served, otherwise the zero matrix would provide a possible solution with zero cost
- thus we require that every row contains exactly a single one
- we also require that every column contains exactly a single one -> the truck cannot be at two locations at the same time, we need an ordering

<img src="images/quadratic_single_truck.PNG" />

In the case of several trucks, the formula for the cost function becomes even more cumbersome.
Here, $x_{ijk}=1$ means that tour number $i$ is served at position $j$ by truck number $k$. 

<img src="images/quadratic_multi_truck.PNG" />

Luckily, python will do these calculations for us...
The important point is, that if we consider the matrix $x_{ij}$ or the tensor $x_{ijk}$ as a $N\times N \times K$-dimensional vector $\vec{x}$ by joining the matrix rows, then we can write the above $f$ as $f(x)= x^TAx$ for some quadratic Matrix $A$ 

# Step 2 The Proof-of-Concept Model in Python

In [16]:
from pathlib import Path
import pickle
from typing import List
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramConverter, LinearEqualityToPenalty
from qiskit_optimization.algorithms import CplexOptimizer

class Location():
    def __init__(self,x:float,y:float):
        self.x=x
        self.y=y   
    
    def print(self):
        print(f"Location at point ({self.x},{self.y})")

class Tour():
    def __init__(self, pickup, delivery, name):
        self.pickupLocation=pickup
        self.deliveryLocation=delivery
        self.name=name

class Truck():
    def __init__(self, depot: Tour, name:str):
        self.depot=depot
        self.name=name
    def print(self):
        print(f"Truck {self.name} at depot {self.depot.name} at {self.depot.pickupLocation.print()}")


class PickUpDeliveryProblem():
    def __init__(self, tours: List, trucks:List, name: str):
        self.name=name
        self.truckList=trucks
        self.numberOfTrucks=len(trucks)
        self.tourList=tours
        self.numberOfTours=len(tours)
        self.adjacencymatrix = []
        self.constraint_matrix = []
        self.cost_matrix = []
        self.constraint_vector = [] 
        
        self.generate_adjacency_matrix()
        self.generate_constraint_matrix()
        self.generate_cost_matrix()
        self.generate_constraint_vector()
    
    # any sensible distance could be used here
    def distance(self, a:Location, b:Location):
        return abs(a.x-b.x) + abs(a.y-b.y)
    
    # connects the endpoint of the first tour with the starting point of the second tour
    def leerkilometer(self, a:Tour, b:Tour):
        return self.distance(a.deliveryLocation, b.pickupLocation)

    # This is the distance matrix D_ij from above
    def generate_adjacency_matrix(self) -> np.ndarray:
        self.adjacencymatrix = np.zeros((self.numberOfTours,self.numberOfTours))
        for count1, tour1 in enumerate(self.tourList):
            for count2, tour2 in enumerate(self.tourList):
                self.adjacencymatrix[count1][count2]=self.leerkilometer(tour1, tour2)

    # This is the matrix A from above for the formulation as a quadratic problem
    def generate_cost_matrix(self)-> np.ndarray:
        n=self.numberOfTours
        self.cost_matrix = np.zeros((n*n*self.numberOfTrucks,n*n*self.numberOfTrucks))
        for truck in range(0,self.numberOfTrucks):
            for i in range(n*n*truck, n*n*(truck +1)):
                tour_i,time_i = divmod(i,n)
                tour_i= tour_i - n*truck

                # Leerfahrten von/zum Depot
                if time_i==0:
                    self.cost_matrix[i][i]=self.leerkilometer(self.truckList[truck].depot, self.tourList[tour_i])
                if time_i==n-1:
                    self.cost_matrix[i][i]=self.leerkilometer(self.tourList[tour_i],self.truckList[truck].depot)

                # Leerfahrten innerhalb mehrerer Touren eines LKW
                for j in range(i,n*n*(truck +1)):
                    tour_j,time_j = divmod(j,n)
                    tour_j = tour_j - n*truck
                    if time_j == (time_i+1):
                        self.cost_matrix[i][j]=self.adjacencymatrix[tour_i][tour_j]
                    if time_i == (time_j+1):
                        self.cost_matrix[i][j]=self.adjacencymatrix[tour_j][tour_i]

                # Tourende eines LKW und Touranfang eines anderen mit entsprechenden Depot-Leerfahrten  
                for j in range(n*n*(truck +1),n*n*self.numberOfTrucks):
                    truck_j, _ = divmod(j,n*n)  
                    tour_j,time_j = divmod(j,n)
                    tour_j = tour_j - n*truck_j
                    if time_j == (time_i+1):
                        self.cost_matrix[i][j]=self.leerkilometer(self.tourList[tour_i],self.truckList[truck].depot) + self.leerkilometer(self.truckList[truck_j].depot,self.tourList[tour_j])
                    if time_i == (time_j+1):
                        self.cost_matrix[i][j]=self.leerkilometer(self.tourList[tour_j],self.truckList[truck_j].depot) + self.leerkilometer(self.truckList[truck].depot,self.tourList[tour_i])

    # here the constraint matrix C to ensure we have an actual permutation matrix is generated
    def generate_constraint_matrix(self)-> np.ndarray:
        n=self.numberOfTours
        self.constraint_matrix = np.zeros((2*n,n*n*self.numberOfTrucks))
        for row_index in range(0, n):
            for truck in range(0, self.numberOfTrucks):
                offset = row_index * n
                self.constraint_matrix[row_index, offset + n*n*truck :offset + n + n*n*truck] = 1

        for row_index in range(n, 2* n):
            offset = row_index - self.numberOfTours
            for col_index in range(0, n * self.numberOfTrucks):
                self.constraint_matrix[row_index, n * col_index + offset] = 1
    
    # here the constraint righthandside e to ensure we have an actual permutation matrix is generated: Cx=e
    def generate_constraint_vector(self)-> np.ndarray:
        self.constraint_vector = np.ones(2* self.numberOfTours)


    # this function translates A, C and e into the quadratic problem for x
    def generate_Quadratic_Program(self, name:str) -> QuadraticProgram:

        qp=QuadraticProgram(name)
        for truck in self.truckList:
            for tour in self.tourList:
                qp.binary_var_list(
                    keys=[ f"{tour.name}_at_Position_{t}_served_by_{truck.name}" for t in range(1, self.numberOfTours +1 ) ],
                    name=""
                )

        
        for row_index in range(0, self.constraint_matrix.shape[0]):
            qp.linear_constraint(
                linear=self.constraint_matrix[row_index, :],
                rhs=self.constraint_vector[row_index],
                sense="==",
                name = f"row_{row_index} sums to one")

        qp.minimize(quadratic = self.cost_matrix)
            
        return qp



**An Example with a single truck**

In [17]:

class PDPExample1(PickUpDeliveryProblem):
    def __init__(self):
        trucklist=[]
        depot=Tour(Location(0,0),Location(0,0),"depot1")
        trucklist.append(Truck(depot, "truck1"))
        
        tourlist=[]
        tourlist.append(Tour(Location(2,3), Location(3,2), "tour1"))
        tourlist.append(Tour(Location(0,0), Location(2,3), "tour2"))
        tourlist.append(Tour(Location(3,3), Location(0,4), "tour3"))
        super().__init__(tourlist, trucklist, "1Truck_3Tours")


pdp1=PDPExample1()

<img src="images\one_truck_problem2.png" />

In [18]:
print(pdp1.cost_matrix)

[[5. 2. 0. 0. 5. 0. 0. 1. 0.]
 [0. 0. 2. 0. 0. 5. 3. 0. 1.]
 [0. 0. 5. 0. 0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 5. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 5. 4. 0. 1.]
 [0. 0. 0. 0. 0. 5. 0. 4. 0.]
 [0. 0. 0. 0. 0. 0. 6. 4. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 4.]
 [0. 0. 0. 0. 0. 0. 0. 0. 4.]]


In [19]:
pdp1=PDPExample1()
print(pdp1.constraint_matrix)

[[1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [1. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1.]]


In [20]:
pdp1=PDPExample1()
qp=pdp1.generate_Quadratic_Program("QP")


#print(qp)
cplex_optimizer = CplexOptimizer()

qcio_minimization_result = cplex_optimizer.solve(qp)

print("minimum point: p_min = ", qcio_minimization_result.x)
print("minimum value: f_1(p_min) = ", qcio_minimization_result.fval)

optimal_value= qcio_minimization_result.fval
print("Q = \n", qp.objective.quadratic.to_array())
#print(qp.binary_var_list)


-- cannot find parameters matching version: 22.1.0.0, using: 20.1.0.0
minimum point: p_min =  [0. 1. 0. 1. 0. 0. 0. 0. 1.]
minimum value: f_1(p_min) =  5.0
Q = 
 [[5. 2. 0. 0. 5. 0. 0. 1. 0.]
 [0. 0. 2. 0. 0. 5. 3. 0. 1.]
 [0. 0. 5. 0. 0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 5. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 5. 4. 0. 1.]
 [0. 0. 0. 0. 0. 5. 0. 4. 0.]
 [0. 0. 0. 0. 0. 0. 6. 4. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 4.]
 [0. 0. 0. 0. 0. 0. 0. 0. 4.]]


The minimum x-value in vector notation [0 1 0 1 0 0 0 0 1]
translates to the permutation matrix:

$\left( \begin{array}{rrr}
0 & 1 & 0 \\ 
1 & 0 & 0 \\
0 & 0 & 1 \\ 
\end{array}\right)$
 This means we start with tour 2, then tour 1, then tour 3.
  

<img src="images\one_truck_problem2.PNG" />

**An Example with two trucks and three tours** 

In [21]:
class PDPExample2(PickUpDeliveryProblem):
    def __init__(self):
        trucklist=[]
        depot1=Tour(Location(4,2),Location(4,2),"depot1")
        trucklist.append(Truck(depot1, "truck1"))
        depot2=Tour(Location(0,0),Location(0,0),"depot2")
        trucklist.append(Truck(depot2, "truck2"))
        
        tourlist=[]
        tourlist.append(Tour(Location(0,1), Location(2,1), "tour1"))
        tourlist.append(Tour(Location(4,3), Location(2,3), "tour2"))
        tourlist.append(Tour(Location(3,0), Location(1,0), "tour3"))
        super().__init__(tourlist, trucklist, "2Trucks_3Tours")

pdp2=PDPExample2()

<img src="images/two_trucks_problem5.png" />

In [22]:
print(pdp2.cost_matrix)

[[ 5.  2.  0.  0.  4.  0.  0.  2.  0.  0.  4.  0.  0. 10.  0.  0.  6.  0.]
 [ 0.  0.  2.  4.  0.  4.  2.  0.  2.  8.  0.  4. 10.  0. 10.  6.  0.  6.]
 [ 0.  0.  3.  0.  4.  0.  0.  2.  0.  0.  8.  0.  0. 10.  0.  0.  6.  0.]
 [ 0.  0.  0.  1.  2.  0.  0.  4.  0.  0.  4.  0.  0. 10.  0.  0.  6.  0.]
 [ 0.  0.  0.  0.  0.  2.  6.  0.  4.  4.  0.  4.  6.  0. 10.  2.  0.  6.]
 [ 0.  0.  0.  0.  0.  3.  0.  6.  0.  0.  4.  0.  0.  6.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  0.  3.  2.  0.  0.  6.  0.  0. 12.  0.  0.  8.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  2.  6.  0.  6.  8.  0. 12.  4.  0.  8.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  5.  0.  6.  0.  0.  8.  0.  0.  4.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  2.  0.  0.  4.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  4.  0.  4.  2.  0.  2.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0.  4.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  7.  2.  0.  0.  4.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [23]:
print(pdp2.constraint_matrix)

[[1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.]]


In [24]:
qp=pdp2.generate_Quadratic_Program("QP")
cplex_optimizer = CplexOptimizer()
qcio_minimization_result = cplex_optimizer.solve(qp)

print("minimum point: p_min = ", qcio_minimization_result.x)
print("minimum value: f_1(p_min) = ", qcio_minimization_result.fval)

optimal_value= qcio_minimization_result.fval
print(qp.binary_var_list)


-- cannot find parameters matching version: 22.1.0.0, using: 20.1.0.0
minimum point: p_min =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
minimum value: f_1(p_min) =  8.0
<bound method QuadraticProgram.binary_var_list of \ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: QP

Minimize
 obj: [ 10 tour1_at_Position_1_served_by_truck1^2
      + 4 tour1_at_Position_1_served_by_truck1*tour1_at_Position_2_served_by_truck1
      + 8 tour1_at_Position_1_served_by_truck1*tour2_at_Position_2_served_by_truck1
      + 4 tour1_at_Position_1_served_by_truck1*tour3_at_Position_2_served_by_truck1
      + 8 tour1_at_Position_1_served_by_truck1*tour1_at_Position_2_served_by_truck2
      + 20 tour1_at_Position_1_served_by_truck1*tour2_at_Position_2_served_by_truck2
      + 12 tour1_at_Position_1_served_by_truck1*tour3_at_Position_2_served_by_truck2
      + 4 tour1_at_Position_2_served_by_truck1*tour1_at_Position_3_served_by_truck1
      + 8 tour1_at_Position_2_serve

The minimal x-value of [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]

translates into a partition of a permutation matrix into two matrices, namely 


$\left( \begin{array}{rrr}
0 & 0 & 0 \\ 
1 & 0 & 0 \\
0 & 0 & 0 \\ 
\end{array}\right)$

$\left( \begin{array}{rrr}
0 & 1 & 0 \\ 
0 & 0 & 0 \\
0 & 0 & 1 \\ 
\end{array}\right)$


 meaning tour 1 and tour 3 are served (in that order) by truck 2
 while tour 2 is served by truck number 1. 

<img src="images/two_trucks_problem5.png" />

# Proof of concept Model in Qiskit

In [25]:
from qiskit_optimization.converters import QuadraticProgramConverter, LinearEqualityToPenalty

class Converter(QuadraticProgramConverter):
    def __init__(self, penalty: float=None) -> None:
        super().__init__()
        self._penalty = penalty
        self.linear_equality_to_penalty_converter = LinearEqualityToPenalty(penalty)

    def convert(self, quadratic_program: QuadraticProgram) -> QuadraticProgram:
        return self.linear_equality_to_penalty_converter.convert(quadratic_program)
    
    def interpret(self, x) -> np.ndarray:
        return self.linear_equality_to_penalty_converter.interpret()

qp=pdp2.generate_Quadratic_Program("QP")
penalty=10
converter=Converter(penalty=penalty)
qubo=converter.convert(qp)


In [26]:
from qiskit.providers.aer import AerSimulator
from qiskit.utils import QuantumInstance
from qiskit.opflow import CircuitSampler

aer_simulator = AerSimulator()
quantum_instance = QuantumInstance(aer_simulator, shots=8000)

aer_simulator_statevector = AerSimulator(method="statevector")
quantum_instance_statevector = QuantumInstance(aer_simulator_statevector)

circuit_sampler = CircuitSampler(quantum_instance_statevector)

In [27]:
from qiskit.providers.aer import AerSimulator
from qiskit.utils import QuantumInstance
from qiskit.opflow import CircuitSampler
from qiskit.utils import algorithm_globals

algorithm_globals.massive=True
aer_simulator = AerSimulator()
quantum_instance = QuantumInstance(aer_simulator, shots=8000)

aer_simulator_qasm = AerSimulator(method="automatic")
quantum_instance_qasm = QuantumInstance(aer_simulator_qasm)

circuit_sampler = CircuitSampler(quantum_instance_qasm)

In [28]:
class PDPExample3(PickUpDeliveryProblem):
    def __init__(self):
        trucklist=[]
        depot1=Tour(Location(4,2),Location(4,2),"depot1")
        trucklist.append(Truck(depot1, "truck1"))
        depot2=Tour(Location(0,0),Location(0,0),"depot2")
        trucklist.append(Truck(depot2, "truck2"))
        
        tourlist=[]
        tourlist.append(Tour(Location(0,1), Location(2,1), "tour1"))
        tourlist.append(Tour(Location(4,3), Location(2,3), "tour2"))
        super().__init__(tourlist, trucklist, "2Trucks_2Tours")

pdp3=PDPExample3()

qp=pdp1.generate_Quadratic_Program("QP")
penalty=10
converter=Converter(penalty=penalty)
qubo=converter.convert(qp)

In [29]:
from ast import And
from qiskit.circuit.library.n_local import QAOAAnsatz
from qiskit.opflow import CircuitStateFn, OperatorStateFn, VectorStateFn, DictStateFn, CircuitSampler

import plotly.graph_objects as go
import pandas as pd


def generate_bitstrings(number_bits: int):
    return [bin(k)[2:].zfill(number_bits) for k in range(2**number_bits)]

def generate_bitarrays(number_bits: int):
    return [np.array(list(bin(k)[2:].zfill(number_bits))[::-1], dtype=int) for k in range(2**number_bits)]

number_binary_variables = qubo.get_num_binary_vars()

ising, ising_offset = qubo.to_ising()
number_of_shots=8000
qaoa_reps=1
simulator_with_shot_noise = False

betas=[1]
gammas=[3]

#%%
qaoa_ansatz = QAOAAnsatz(cost_operator=ising, reps=qaoa_reps)
wave_function = CircuitStateFn(qaoa_ansatz)
observable = OperatorStateFn(ising, is_measurement=True)
expectation_operator = observable @ wave_function

aer_simulator_statevector = AerSimulator(method="statevector", shots=number_of_shots)
quantum_instance = QuantumInstance(aer_simulator_statevector)
circuit_sampler = CircuitSampler(quantum_instance, statevector=not simulator_with_shot_noise)

def run_qaoa(parameter_values):
    "parameter_values is expected to be of the form [beta_0, beta_1, ..., gamma_0, gamma_1, ...]"
    parameter_bindings = dict(zip(qaoa_ansatz.parameters, parameter_values))
    wave_function_sampled = circuit_sampler.convert(wave_function, params=parameter_bindings)
    return wave_function_sampled

#%%
# Be careful: the data type of wave_function_sampled depends if shot noise is present
wave_function_sampled = run_qaoa([*betas, *gammas])

if isinstance(wave_function_sampled, DictStateFn): # data type returned in case with shot noise
    probabilities = np.real(wave_function_sampled.to_matrix()**2)
elif isinstance(wave_function_sampled, VectorStateFn): # data type returned in case without shot noise
    probabilities = np.abs(wave_function_sampled.to_matrix())**2
assert np.isclose(probabilities.sum(), 1), "Error: probabilities don't sum up to 1."

df_simulation = pd.DataFrame(index=generate_bitstrings(number_binary_variables))
df_simulation['probability'] = probabilities

#%%
df_further_info = pd.DataFrame(index=generate_bitstrings(number_binary_variables))
df_further_info["bitarray"] = generate_bitarrays(number_binary_variables)
df_further_info["cost"] = df_further_info["bitarray"].apply(qp.objective.evaluate)
df_further_info["is_feasible"] = df_further_info["bitarray"].apply(qp.is_feasible)
df_further_info["is_optimal"] = (df_further_info["cost"] - optimal_value).apply(lambda x: np.isclose(x, 0)) 
df_further_info["is_optimal_and_feasible"] = df_further_info["is_optimal"] & df_further_info["is_feasible"]



#%%
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_simulation.index,
    y=df_simulation["probability"],
    line_color="black",
    line_width=1.5,
    name=f"simulation,<br>shot noise: {simulator_with_shot_noise}"
))
fig.add_trace(go.Scatter(
    x=df_simulation[df_further_info["is_optimal_and_feasible"]].index,
    y=df_simulation[df_further_info["is_optimal_and_feasible"]]["probability"],
    mode="markers",
    marker_symbol="arrow-down",
    marker_size=12,
    marker_color="red",
    name="optimal solution"
))
fig.add_trace(go.Scatter(
    x=df_simulation[df_further_info["is_feasible"]].index,
    y=df_simulation[df_further_info["is_feasible"]]["probability"],
    mode="markers",
    marker_symbol="cross",
    marker_size=12,
    marker_color="blue",
    name="feasible solution"
))
fig.update_xaxes(
    tickfont_size=8,
)
fig.update_yaxes(
    title="probability",
    title_font_size=12
)
title_betas = "".join([f"{beta:.3f}, " for beta in betas])
title_gammas = "".join([f"{gamma:.3f}, " for gamma in gammas])
fig.update_layout(
    title=f"example_3tours, penalty: {penalty:.3f}; QAOA: p={qaoa_reps}<br>" \
        + "betas: " + title_betas + "gammas: " + title_gammas,
    title_font_size=12
)

In [30]:
print(df_further_info.sort_values(by="is_feasible"))

                              bitarray  cost  is_feasible  is_optimal  \
000000000  [0, 0, 0, 0, 0, 0, 0, 0, 0]   0.0        False       False   
101011101  [1, 0, 1, 1, 1, 0, 1, 0, 1]  35.0        False       False   
101011100  [0, 0, 1, 1, 1, 0, 1, 0, 1]  25.0        False       False   
101011011  [1, 1, 0, 1, 1, 0, 1, 0, 1]  36.0        False       False   
101011010  [0, 1, 0, 1, 1, 0, 1, 0, 1]  24.0        False       False   
...                                ...   ...          ...         ...   
100001010  [0, 1, 0, 1, 0, 0, 0, 0, 1]   5.0         True       False   
100010001  [1, 0, 0, 0, 1, 0, 0, 0, 1]  15.0         True       False   
001010100  [0, 0, 1, 0, 1, 0, 1, 0, 0]  15.0         True       False   
010001100  [0, 0, 1, 1, 0, 0, 0, 1, 0]   9.0         True       False   
001100010  [0, 1, 0, 0, 0, 1, 1, 0, 0]  19.0         True       False   

           is_optimal_and_feasible  
000000000                    False  
101011101                    False  
101011100   

**The same result for p=1 without a fixed depot**

<img src="plots/Simulationen/example_3Tours_p=1.PNG" />

**Simulation for p=2**

<img src="plots/Simulationen/example_3Tours_p=2.PNG" />

**p=3**

<img src="plots/Simulationen/example_3Tours_p=3.PNG" />

**p=3 with shot noise**

<img src="plots/Simulationen/example_3Tours_p=3_shotNoise.png" />

In [32]:
import qiskit
pd.show_versions()


INSTALLED VERSIONS
------------------
commit           : 945c9ed766a61c7d2c0a7cbb251b6edebf9cb7d5
python           : 3.9.13.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19045
machine          : AMD64
processor        : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : de_DE.cp1252

pandas           : 1.3.4
numpy            : 1.21.4
pytz             : 2021.3
dateutil         : 2.8.2
pip              : 22.1.2
setuptools       : 58.1.0
Cython           : 0.29.25
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : 4.6.4
html5lib         : 1.1
pymysql          : None
psycopg2         : None
jinja2           : 3.0.3
IPython          : 7.30.1
pandas_datareader: None
bs4              : None
bottleneck       : None
fsspec           :

**Copyright 2023 Florian Knäble, Fraunhofer IAO**

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.